<a href="https://colab.research.google.com/github/huerd/GPA659-E2020/blob/howard%2Fproj/_project/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# select your Kaggle API kaggle.json
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [2]:
# updates kaggle version
!pip install --upgrade --force-reinstall --no-deps kaggle

from __future__ import print_function

# uncomment below to use version 1.x
#%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import zipfile
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=558b0efa6c2396f7f0b73bed29a6e89e56c6601eddc9600fab353f3a20088897
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


Download and extract data

In [3]:
# make sure to create a kaggle account, get it verified (phone number and all)
# then go accept terms here : https://www.kaggle.com/c/dogs-vs-cats

# zip should be downloaded to /content/
!kaggle competitions download -c dogs-vs-cats
zip_ref = zipfile.ZipFile("dogs-vs-cats.zip", 'r')
zip_ref.extractall()
zip_ref.close()

# don't expand the folders unless you want colab to crash on you
# file format is 1.jpg, 2.jpg, etc
unzipTest1 = zipfile.ZipFile("test1.zip", 'r')
unzipTest1.extractall()
unzipTest1.close()

# file format is cat.0.jpg, dog.2.jpg, etc
unzipTrain1 = zipfile.ZipFile("train.zip", 'r')
unzipTrain1.extractall()
unzipTrain1.close()

100% 812M/812M [00:09<00:00, 109MB/s] 
100% 812M/812M [00:09<00:00, 86.8MB/s]


In [12]:
import os
import pandas
import numpy

trainingDataset = os.listdir("train")
testingDataset = os.listdir("test1")

results = []

totalImages = 0;

# loop through our trainingDataset, one file at a time
for eachImage in trainingDataset:
  # recall that our trainingDataset file format is [animal].[num].jpg
  petType = eachImage.split('.')[0]
  if petType == 'cat':
    results.append("cat")
  else:
    results.append("dog")

tableResults = pandas.DataFrame({
    'imageName': trainingDataset,
    'petType': results
})


Convert our data to Numpy tables

In [20]:
# using https://www.kaggle.com/lgmoneda/from-image-files-to-numpy-arrays as reference

print("# of Training data = {0} ".format(len(trainingDataset)) + "images")
print("# of Testing data = {0} ".format(len(testingDataset)) + "images")



# of Training data = 25000 images
# of Testing data = 12500 images
Image examples: 
Image examples: 
